import itertoolsSee https://rosettacode.org/wiki/Ordered_partitions#Python  - the 2nd algo

In [9]:
import itertools

from coppertop.pipe import *
from dm.core.types import pylist, index
from dm.testing import check, equals
from dm.core import first, count, drop, collect, prependTo, join, joinAll, take, sum, unpack
from dm.pp import PP

In [10]:
def partitionsBruteForce(cards, handSizes):
    slices = []
    s1 = 0
    for handSize in handSizes:
        s2 = s1 + handSize
        slices.append((s1, s2))
        s1 = s2
    perms = filter(
        lambda perm: groupsInOrder(perm, slices),
        itertools.permutations(cards, len(cards))
    )
    return tuple(perms)

def groupsInOrder(xs, slices):
    for s1, s2 in slices:
        if not isAsc(xs[s1:s2]): return False
    return True

def isAsc(xs):
    p = xs[0]
    for n in xs[1:]:
        if n <= p: return False
        p = n
    return True

In [11]:
partitionsBruteForce((1,2,3,4,5), (2,3))

((1, 2, 3, 4, 5),
 (1, 3, 2, 4, 5),
 (1, 4, 2, 3, 5),
 (1, 5, 2, 3, 4),
 (2, 3, 1, 4, 5),
 (2, 4, 1, 3, 5),
 (2, 5, 1, 3, 4),
 (3, 4, 1, 2, 5),
 (3, 5, 1, 2, 4),
 (4, 5, 1, 2, 3))

In [12]:
# partitions :: [Int] -> [[[Int]]]
def partitionsML(xs, sizes):
    n = sum(sizes)

    def go(xs, n, sizes):
        return [
            [l] + r
            for (l, rest) in choose(xs)(n)(sizes[0])
            for r in go(rest, n - sizes[0], sizes[1:])
        ] if sizes else [[]]

    return go(xs, n, sizes)


# choose :: [Int] -> Int -> Int -> [([Int], [Int])]
def choose(xs):
    '''(m items chosen from n items, the rest)'''

    def go(xs, n, m):
        f = cons(xs[0])
        choice = choose(xs[1:])(n - 1)
        return [([], xs)] if 0 == m else (
            [(xs, [])] if n == m else (
                    [first(f)(xy) for xy in choice(m - 1)] +
                    [second(f)(xy) for xy in choice(m)]
            )
        )

    return lambda n: lambda m: go(xs, n, m)


# cons :: a -> [a] -> [a]
def cons(x):
    '''Construction of a list from x as head, and xs as tail.'''
    return lambda xs: [x] + xs


# first :: (a -> b) -> ((a, c) -> (b, c))
def first(f):
    '''A simple function lifted to a function over a tuple, with f applied only the first of two values.'''
    return lambda xy: (f(xy[0]), xy[1])


# second :: (a -> b) -> ((c, a) -> (c, b))
def second(f):
    '''A simple function lifted to a function over a tuple, with f applied only the second of two values.'''
    return lambda xy: (xy[0], f(xy[1]))

In [13]:
partitionsML([1,2,3,4,5], (2,3))

[[[1, 2], [3, 4, 5]],
 [[1, 3], [2, 4, 5]],
 [[1, 4], [2, 3, 5]],
 [[1, 5], [2, 3, 4]],
 [[2, 3], [1, 4, 5]],
 [[2, 4], [1, 3, 5]],
 [[2, 5], [1, 3, 4]],
 [[3, 4], [1, 2, 5]],
 [[3, 5], [1, 2, 4]],
 [[4, 5], [1, 2, 3]]]

In [14]:
%timeit  partitionsML(list(range(13)), [5,4,4]) >> count >> PP

90090
90090
90090
90090
90090
90090
90090
90090
330 ms ± 3.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
from dm.core import first

In [19]:
@coppertop(style=binary)
def takeDrop(xs, s):
    return xs[:s], xs[s:]

In [20]:
@coppertop(style=binary)
def partitionsCpt(es, sizes: pylist) -> pylist:
    sizes >> sum >> check >> equals >> (es >> count)
    return _partitionsCpt(list(es), es >> count, sizes)

@coppertop
def _partitionsCpt(es: pylist, n: index, sizes: pylist) -> pylist:
    if not sizes: return [[]]
    return es >> _combRestCpt(_, n, sizes >> first) \
        >> collect >> (unpack(lambda x, y:
            _partitionsCpt(y, n - (sizes >> first), sizes >> drop >> 1)
                >> collect >> (lambda partitions:
                    x >> prependTo >> partitions
                )
        )) \
        >> joinAll

@coppertop
def _combRestCpt(es: pylist, n: index, m: index) -> pylist:
    '''answer [m items chosen from n items, the rest]'''
    if m == 0: return [([], es)]
    if m == n: return [(es, [])]
    s1, s2 = es >> takeDrop >> 1
    return \
        (s2 >> _combRestCpt(_, n - 1, m - 1) >> collect >> unpack(lambda x, y: (s1 >> join >> x, y))) \
        >> join >> \
        (s2 >> _combRestCpt(_, n - 1, m) >> collect >> unpack(lambda x, y: (x, s1 >> join >> y)))

In [21]:
[1,2,3,4,5] >> partitionsCpt >> [2,3]

[[[1, 2], [3, 4, 5]],
 [[1, 3], [2, 4, 5]],
 [[1, 4], [2, 3, 5]],
 [[1, 5], [2, 3, 4]],
 [[2, 3], [1, 4, 5]],
 [[2, 4], [1, 3, 5]],
 [[2, 5], [1, 3, 4]],
 [[3, 4], [1, 2, 5]],
 [[3, 5], [1, 2, 4]],
 [[4, 5], [1, 2, 3]]]

In [22]:
%timeit range(13) >> partitionsCpt >> [5,4,4] >> count >> PP

90090
90090
90090
90090
90090
90090
90090
90090
5.11 s ± 54.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
@coppertop(style=binary)
def partitionsCptFaster(xs, sizes: pylist) -> pylist:
    sizes >> sum >> check >> equals >> (xs >> count)
    return _partitionsCptFaster(list(xs), xs >> count, sizes)

@coppertop
def _partitionsCptFaster(xs: pylist, n: index, sizes: pylist) -> pylist:
    if not sizes: return [[]]
    return _combRestCptFaster(xs, n, sizes[0]) \
        >> collect >> (unpack(lambda comb, rest:
            _partitionsCptFaster(rest, n - sizes[0], sizes[1:])
                >> collect >> (lambda partitions:
                    [comb] + partitions
                )
        )) \
        >> joinAll

@coppertop
def _combRestCptFaster(xs: pylist, n: index, m: index) -> pylist:
    '''answer [m items chosen from n items, the rest]'''
    if m == 0: return [([], xs)]
    if m == n: return [(xs, [])]
    s1, s2 = xs[:1], xs[1:]
    return \
        (_combRestCptFaster(s2, n - 1, m - 1) >> collect >> (lambda xy: (s1 + xy[0], xy[1]))) + \
        (_combRestCptFaster(s2, n - 1, m) >> collect >> (lambda xy: (xy[0], s1 + xy[1])))


In [24]:
[1,2,3,4,5] >> partitionsCptFaster >> [2,3]

[[[1, 2], [3, 4, 5]],
 [[1, 3], [2, 4, 5]],
 [[1, 4], [2, 3, 5]],
 [[1, 5], [2, 3, 4]],
 [[2, 3], [1, 4, 5]],
 [[2, 4], [1, 3, 5]],
 [[2, 5], [1, 3, 4]],
 [[3, 4], [1, 2, 5]],
 [[3, 5], [1, 2, 4]],
 [[4, 5], [1, 2, 3]]]

In [25]:
%timeit range(13) >> partitionsCptFaster >> [5,4,4] >> count >> PP

90090
90090
90090
90090
90090
90090
90090
90090
2.61 s ± 19.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
@coppertop(style=binary)
def partitionsCptFasterStill(xs, sizes: pylist) -> pylist:
    sizes >> sum >> check >> equals >> (xs >> count)
    return _partitionsCptFasterStill(list(xs), xs >> count, sizes)

def _partitionsCptFasterStill(xs: pylist, n: index, sizes: pylist) -> pylist:
    if not sizes: return [[]]
    return list(itertools.chain(*map(
        lambda comb_rest: map(
            lambda partitions: [comb_rest[0]] + partitions,
            _partitionsCptFasterStill(comb_rest[1], n - sizes[0], sizes[1:])
        ),
        _combRestCptFasterStill(xs, n, sizes[0])
    )))

def _combRestCptFasterStill(xs: pylist, n: index, m: index) -> pylist:
    '''answer [m items chosen from n items, the rest]'''
    if m == 0: return [([], xs)]
    if m == n: return [(xs, [])]
    s1, s2 = xs[:1], xs[1:]
    return list(itertools.chain(
        map(lambda xy: (s1 + xy[0], xy[1]), _combRestCptFasterStill(s2, n - 1, m - 1)),
        map(lambda xy: (xy[0], s1 + xy[1]), _combRestCptFasterStill(s2, n - 1, m))
    ))

In [27]:
[1,2,3,4,5] >> partitionsCptFasterStill >> [2,3]

[[[1, 2], [3, 4, 5]],
 [[1, 3], [2, 4, 5]],
 [[1, 4], [2, 3, 5]],
 [[1, 5], [2, 3, 4]],
 [[2, 3], [1, 4, 5]],
 [[2, 4], [1, 3, 5]],
 [[2, 5], [1, 3, 4]],
 [[3, 4], [1, 2, 5]],
 [[3, 5], [1, 2, 4]],
 [[4, 5], [1, 2, 3]]]

In [29]:
%timeit range(13) >> partitionsCptFasterStill >> [5,4,4] >> count >> PP

90090
90090
90090
90090
90090
90090
90090
90090
200 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


BONES

```
load dm.core
from dm.core import sum, count, isEmpty, ifTrue:, collect, first, drop, prependTo, joinAll, takeDrop, join
from dm.testing import check, equal

partitions: {{[xs:N1**T1, sizes:N2**count] <:N**N2**N**T1>
    sizes sum check equal (xs count)
    xs _partitions(, xs count, sizes)
}}

_partitions: {[xs:N1**T1, n:count, sizes:N2**count] <:N**N2**N**T1>
    sizes isEmpty ifTrue: [^ (())]
    xs _combRest(, n, sizes first) collect {[a, b]
        _partitions(b, .n - (.sizes first), .sizes drop 1) collect {
            .a prependTo r
        }
    } joinAll
}

_combRest: {[xs:N**T1, n:count, m:count] <:N**(N**T1)*(N**T1)>
    m == 0 ifTrue: [^ ( (() <:N**T1>, xs) )]
    m == n ifTrue: [^ ( (xs, () <:N**T1>) )]
    (comb, rest): xs takeDrop 1
    rest _combRest(, s2 count, m - 1) collect { (.comb join a, b) }    // #1
      join
      (rest _combRest(, s2 count, m) collect { (a, .comb join b) })    // #2
}
```